In [1]:
import pandas as pd
import sqlite3


In [2]:
orders = pd.read_csv("orders.csv")
orders.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [3]:
users = pd.read_json("users.json")
users.head()


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [4]:
conn = sqlite3.connect(":memory:")

with open("restaurants.sql", "r", encoding="utf-8") as f:
    conn.executescript(f.read())

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants.head()


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [5]:
df = orders.merge(users, on="user_id", how="left") \
           .merge(restaurants, on="restaurant_id", how="left")

df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [7]:
df["order_date"] = pd.to_datetime(df["order_date"], dayfirst=True)
df["quarter"] = df["order_date"].dt.to_period("Q")

df.shape   


(10000, 13)

In [8]:
df[df.membership=="Gold"] \
  .groupby("city")["total_amount"] \
  .sum() \
  .idxmax()


'Chennai'

In [9]:
df.groupby("cuisine")["total_amount"] \
  .mean() \
  .idxmax()


'Mexican'

In [10]:
(df.groupby("user_id")["total_amount"].sum() > 1000).sum()


np.int64(2544)

In [11]:
bins = [0, 3.5, 4.0, 4.5, 5.0]
labels = ["3.0–3.5", "3.6–4.0", "4.1–4.5", "4.6–5.0"]

df["rating_band"] = pd.cut(df["rating"], bins=bins, labels=labels, include_lowest=True)

df.groupby("rating_band")["total_amount"] \
  .sum() \
  .idxmax()


C:\Users\sama\AppData\Local\Temp\ipykernel_2572\3962085040.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("rating_band")["total_amount"] \


'4.6–5.0'

In [12]:
df[df.membership=="Gold"] \
  .groupby("city")["total_amount"] \
  .mean() \
  .idxmax()


'Chennai'

In [13]:
df.groupby("cuisine").agg(
    restaurant_count=("restaurant_id","nunique"),
    revenue=("total_amount","sum")
).sort_values("restaurant_count")


,restaurant_count,revenue
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


In [14]:
round((df.membership=="Gold").mean() * 100)


50

In [17]:
df.groupby("restaurant_name").agg(
    avg_order_value=("total_amount","mean"),
    total_orders=("order_id","count")
).query("total_orders < 20") \
 .sort_values("avg_order_value", ascending=False) \
 .head(1)


KeyError: 'restaurant_name'

In [16]:
df.groupby(["membership","cuisine"])["total_amount"] \
  .sum() \
  .idxmax()


('Regular', 'Mexican')

In [18]:
df.groupby("quarter")["total_amount"] \
  .sum() \
  .idxmax()


Period('2023Q3', 'Q-DEC')

In [19]:
df[df.membership=="Gold"].shape[0]


4987

In [20]:
round(df[df.city=="Hyderabad"]["total_amount"].sum())


1889367

In [21]:
df["user_id"].nunique()


2883

In [22]:
round(df[df.membership=="Gold"]["total_amount"].mean(), 2)


np.float64(797.15)

In [23]:
df[df.rating >= 4.5].shape[0]


3374

In [24]:
top_gold_city = df[df.membership=="Gold"] \
                    .groupby("city")["total_amount"] \
                    .sum() \
                    .idxmax()

df[(df.membership=="Gold") & (df.city==top_gold_city)].shape[0]


1337

In [25]:
df.to_csv("final_food_delivery_dataset.csv", index=False)
